# Global Socioeconomic  Analytical Dashboard

In [91]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
from plotly import tools
import plotly.figure_factory as ff

import warnings
warnings.filterwarnings('ignore')

from subprocess import check_output
from PIL import Image
from dash import Dash, html, dash_table, dcc
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd

import dash
import dash_html_components as html
import dash_bootstrap_components as dbc  
from dash import html, dcc, Dash, Input, Output
from dash import dcc
import plotly.express as px
import dash_bootstrap_components as dbc
import math


### Preprocessing child mortality dataset and merging datasets

In [47]:
# Load the initial dataset
initial_df = pd.read_csv("child_mortality2.csv")

# Melt the dataframe to unpivot the year columns
melted_df = pd.melt(initial_df, id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'],
                    var_name='Year', value_name='Value')

# Pivot the dataframe to restructure it
restructured_df = melted_df.pivot_table(index=['Country Name', 'Country Code', 'Year'],
                                        columns='Indicator Name', values='Value').reset_index()

# Rename the columns as needed
restructured_df.columns.name = None  # Remove the name of the columns index
restructured_df = restructured_df.rename(columns={'Country Name': 'Entity', 'Country Code': 'Code'})

#year as integer
restructured_df['Year'] = restructured_df['Year'].astype(int)
# Display the transformed dataframe
print(restructured_df)

            Entity Code  Year  Mortality rate, under-5 (per 1,000 live births)
0      Afghanistan  AFG  1960                                            357.3
1      Afghanistan  AFG  1961                                            351.7
2      Afghanistan  AFG  1962                                            345.8
3      Afghanistan  AFG  1963                                            340.2
4      Afghanistan  AFG  1964                                            334.8
...            ...  ...   ...                                              ...
12660     Zimbabwe  ZWE  2017                                             56.2
12661     Zimbabwe  ZWE  2018                                             53.7
12662     Zimbabwe  ZWE  2019                                             52.7
12663     Zimbabwe  ZWE  2020                                             51.8
12664     Zimbabwe  ZWE  2021                                             49.5

[12665 rows x 4 columns]


In [48]:
df = pd.read_csv("life-expectancy-vs-gdp-per-capita.csv")

In [49]:
merged_df = pd.merge(df, restructured_df[['Entity', 'Year', 'Mortality rate, under-5 (per 1,000 live births)']], 
                     on=['Entity', 'Year'], how='left')

In [50]:
# Assuming df is your DataFrame
merged_df = merged_df.drop(0).reset_index(drop=True)

## Figure 1  = Map showing socialeconomic figures globally overtime

### adding dropdown menu

In [51]:
# Define the dropdown options
dropdown_options = [{'label': col, 'value': col} for col in df.columns]

In [52]:
def update_map(selected_column):
    minimum = df[selected_column].min()
    maximum = df[selected_column].max()

    fig_map = px.choropleth(df,
                            locations="Country",
                            locationmode='country names',
                            color=selected_column,
                            range_color=(minimum, maximum),
                            hover_name="Country",
                            hover_data={"Life expectancy": True, "GDP per capita": True,
                                        "Mortality rate, under-5 (per 1,000 live births)": True},
                            color_continuous_scale='Edge_r',
                            labels={'number': selected_column},
                            animation_frame='Year')

    # Update layout of the map
    fig_map.update_layout(
        title_text='Global Life Expectancy from 1950 - 2021',
        title_font=dict(size=24, color="black"),
        geo=dict(showframe=True, showcoastlines=True),
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        margin=dict(l=2, r=10, t=37, b=10),  # Reduce margins
        coloraxis_colorbar=dict(len=0.75, x=0.5, y=-0.2, orientation='h'),  # Adjust position and length of colorbar
        coloraxis_colorbar_ticks="outside",  # Place colorbar ticks outside the colorbar
        coloraxis_colorbar_tickfont=dict(size=12),  # Adjust colorbar tick font size
        legend=dict(font=dict(size=10)),
        height=600, width=1000,  # Adjust plot height
        sliders=dict(
            x=0.1,  # Adjust x position of the slider (0 is left, 1 is right)
            y=0.5,  # Adjust y position of the slider (0 is bottom, 1 is top)
        )
    )

    return fig_map

## Figure 2 - Life expectancy vs GDP

In [53]:
df1 = merged_df.copy()
# Filter out rows where the year is below 1950

In [54]:
df1.head()

,Entity,Code,Year,Period life expectancy at birth - Sex: all - Age: 0,GDP per capita,417485-annotations,Population (historical estimates),Continent,"Mortality rate, under-5 (per 1,000 live births)"
0,Afghanistan,AFG,1950,27.7275,1156.0,NaN,7480464.0,NaN,NaN
1,Afghanistan,AFG,1951,27.9634,1170.0,NaN,7571542.0,NaN,NaN
2,Afghanistan,AFG,1952,28.4456,1189.0,NaN,7667534.0,NaN,NaN
3,Afghanistan,AFG,1953,28.9304,1240.0,NaN,7764549.0,NaN,NaN
4,Afghanistan,AFG,1954,29.2258,1245.0,NaN,7864289.0,NaN,NaN


In [55]:
#unique_values = df1['Entity'].unique()
#print(unique_values)

i want to plot all the countries, plus the continents -which are: \
Africa, North America, Europe, South America, Asia, Oceania.\
However, North America and South America are missing from the dataset.
Also, I don't have GDP information for Asia and Oceania.

In [56]:
df1 = merged_df.copy()
# List of specified regions
specified_regions = ['Africa', 'Northen America', 'Europe', 'South America', 'Asia', 'Oceania']
# Filter rows where 'Entity' column matches specified regions and set 'Code' column to "CON"

df1.loc[df1['Entity'].isin(specified_regions), 'Code'] = "CON"
#Remove rows with empty values in column 'Code'
df1 = df1.dropna(subset=['Code'])
df1 = df1[(df1['Year'] >= 1980) & (df1['Year'] <= 2018)]
df1.rename(columns={'Entity': 'Country'}, inplace=True)
df1.rename(columns={'Period life expectancy at birth - Sex: all - Age: 0': 'Life expectancy'}, inplace=True)

## adding gdp data for continents:

In [57]:
gdpdata = pd.read_csv("gdp_data_csv.csv")

In [58]:
gdpdata.head()

,Country,1980,1981,1982,1983,1984,1985,1986,1987,1988,...,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028
0,Afghanistan,no data,no data,no data,no data,no data,no data,no data,no data,no data,...,586.204,611.268,443.385,no data,no data,no data,no data,no data,no data,no data
1,Albania,728.359,817.734,824.542,815.529,788.332,788.801,855.724,832.245,805.046,...,5345.058,5278.986,6259.762,6657.637,8057.49,8877.337,9280.572,9818.569,10470.18,11187.728
2,Algeria,2268.607,2305.505,2254.328,2316.679,2432.717,2753.697,2698.915,2705.111,2143.742,...,3953.402,3321.601,3659.709,4306.82,4874.706,5130.36,5243.056,5335.671,5355.284,5365.19
3,Andorra,no data,no data,no data,no data,no data,no data,no data,no data,no data,...,40688.491,36973.845,41806.876,41084.874,44107.317,45642.103,46215.307,46546.578,46713.341,46920.949
4,Angola,802.627,731.427,712.576,723.654,747.325,817.068,743.735,828.958,875.526,...,2612.246,1709.283,2169.648,3438.147,2550.001,2452.737,2483.027,2507.49,2570.826,2622.394


In [59]:
# Melt the DataFrame to have the years as a separate column
melted_gdpdata = pd.melt(gdpdata, id_vars=["Country"], var_name="Year", value_name="GDP_per_capita")

# Convert the "GDP_per_capita" column to float and replace "no data" with NaN
melted_gdpdata['GDP_per_capita'] = pd.to_numeric(melted_gdpdata['GDP_per_capita'], errors='coerce')

melted_gdpdata['Year'] = melted_gdpdata['Year'].astype(int)

# Sort the dataset by the "Country" columns
melted_gdpdata = melted_gdpdata.sort_values(by=['Country', 'Year']).reset_index(drop=True)

# Filter rows where "Country" column is not null
melted_gdpdata = melted_gdpdata[melted_gdpdata['Country'].notnull()]

# Filter out rows in sorted_gdpdata where 'Country' is not in df1['Country']
#melted_gdpdata = melted_gdpdata[melted_gdpdata['Country'].isin(df1['Country'])]

# Reset index
melted_gdpdata.reset_index(drop=True, inplace=True)

# Display the resulting DataFrame
melted_gdpdata

,Country,Year,GDP_per_capita
0,ASEAN-5,1980,805.418
1,ASEAN-5,1981,866.530
2,ASEAN-5,1982,885.289
3,ASEAN-5,1983,845.602
4,ASEAN-5,1984,862.837
...,...,...,...
11216,"©IMF, 2023",2024,NaN
11217,"©IMF, 2023",2025,NaN
11218,"©IMF, 2023",2026,NaN
11219,"©IMF, 2023",2027,NaN


In [60]:
###adding oceania and asia

# Duplicate rows where Country is "Asia and Pacific"
asia_pacific_rows = melted_gdpdata[melted_gdpdata['Country'] == 'Asia and Pacific'].copy()

# Replace Country with "Oceania" in the duplicated rows
asia_pacific_rows['Country'] = 'Oceania'

# Concatenate the original dataframe with the duplicated and modified rows
melted_gdpdata = pd.concat([melted_gdpdata, asia_pacific_rows], ignore_index=True)

# Duplicate rows where Country is "Asia and Pacific"
asia_pacific_rows = melted_gdpdata[melted_gdpdata['Country'] == 'Asia and Pacific'].copy()

# Replace Country with "Oceania" in the duplicated rows
asia_pacific_rows['Country'] = 'Asia'

# Concatenate the original dataframe with the duplicated and modified rows
melted_gdpdata = pd.concat([melted_gdpdata, asia_pacific_rows], ignore_index=True)

gdpcontinents = melted_gdpdata[melted_gdpdata['Country'].isin(['Africa (Region)',
                                                               'North America', 'South America',
                                                               'Asia', 'Oceania', 'Europe', 'World'])]


gdpcontinents = gdpcontinents[gdpcontinents['Year'] <= 2018]
gdpcontinents.rename(columns={'GDP_per_capita': 'GDP per capita'}, inplace=True)
gdpcontinents['Country'] = gdpcontinents['Country'].replace({'Africa (Region)': 'Africa'})

gdpcontinents

,Country,Year,GDP per capita
147,Africa,1980,NaN
148,Africa,1981,NaN
149,Africa,1982,NaN
150,Africa,1983,NaN
151,Africa,1984,NaN
...,...,...,...
11304,Asia,2014,6304.746
11305,Asia,2015,6193.753
11306,Asia,2016,6384.362
11307,Asia,2017,6777.094


In [61]:
# Find unique values in the 'Country' column
unique_countries = gdpcontinents['Country'].unique()
print(unique_countries)

['Africa' 'Europe' 'North America' 'South America' 'World' 'Oceania'
 'Asia']


In [62]:
# Merge the two DataFrames
df1 = pd.merge(df1, gdpcontinents, on=['Country', 'Year'], how='left')

# Fill missing values in 'GDP per capita' column of df1 with values from gdpcontinents
df1['GDP per capita'] = df1['GDP per capita_x'].fillna(df1['GDP per capita_y'])

# Drop the redundant columns
df1.drop(['GDP per capita_x', 'GDP per capita_y'], axis=1, inplace=True)


In [63]:
df1.head()

,Country,Code,Year,Life expectancy,417485-annotations,Population (historical estimates),Continent,"Mortality rate, under-5 (per 1,000 live births)",GDP per capita
0,Afghanistan,AFG,1980,39.6181,NaN,12486640.0,NaN,243.8,1019.0
1,Afghanistan,AFG,1981,40.1640,NaN,11155196.0,NaN,237.3,1144.0
2,Afghanistan,AFG,1982,37.7661,NaN,10088290.0,NaN,230.8,1270.0
3,Afghanistan,AFG,1983,38.1865,NaN,9951447.0,NaN,224.2,1347.0
4,Afghanistan,AFG,1984,33.3289,NaN,10243689.0,NaN,217.7,1337.0


### Creating graph

In [64]:
# Create a new column indicating if each data point belongs to one of the specified regions
df1['Region'] = df1['Country'].apply(lambda x: 'Other' if x not in ['Africa', 'North America', 'Europe', 'Asia', 'Oceania', 'World'] else x)

# Create the scatter plot with different colors for the specified regions
fig_scatter = px.scatter(df1, x="GDP per capita", y="Life expectancy",
                         hover_data=["Country", "Mortality rate, under-5 (per 1,000 live births)"], hover_name="Country", animation_frame="Year",
                         trendline="ols", trendline_options=dict(log_x=True),
                         color='Region',title="GDP per capita vs. Life Expectancy Overtime",
                         color_discrete_map={'Other': 'grey'})

# Update opacity specifically for the "Other" colored dots
fig_scatter.update_traces(marker=dict(opacity=0.5), selector=dict(marker=dict(color='grey')))

# Manually set opacity for 'Other' color in color_discrete_map
fig_scatter.for_each_trace(lambda t: t.update(marker=dict(opacity=0.2, size = 4)) if t.marker.color == 'grey' else ())
# Change label in legend
fig_scatter.for_each_trace(lambda t: t.update(name='Countries') if t.marker.color == 'grey' else ())
# Manually set opacity for 'World' color in color_discrete_map
fig_scatter.for_each_trace(lambda t: t.update(marker=dict(opacity=0.8, size = 16)) if t.marker.color != 'grey' else ())

# Add title to the graph
fig_scatter.update_layout(
                            yaxis_title="Life Expectancy (years)",
                            title=dict(font_size=25),
                            xaxis_title="GDP per capita ($)",
                            legend=dict(title_font_size=20,font_size=18),
                            margin=dict(l=10, r=5, t=40, b=0),
                           yaxis=dict(title_font_size=20,tickfont_size=16),
                           xaxis=dict(title_font_size=20,tickfont_size=16),
                            xaxis_title_font=dict(size=20),
                           height=600, width=800)  # Adjust height and width of the plot


#fig_scatter.update_traces(line=dict(color="black"))
fig_scatter.show()

## Figure 3 - Bar chart showing difference in life expectancy for low income vs high income

In [65]:
df2 = merged_df.copy()
df2.head()

,Entity,Code,Year,Period life expectancy at birth - Sex: all - Age: 0,GDP per capita,417485-annotations,Population (historical estimates),Continent,"Mortality rate, under-5 (per 1,000 live births)"
0,Afghanistan,AFG,1950,27.7275,1156.0,NaN,7480464.0,NaN,NaN
1,Afghanistan,AFG,1951,27.9634,1170.0,NaN,7571542.0,NaN,NaN
2,Afghanistan,AFG,1952,28.4456,1189.0,NaN,7667534.0,NaN,NaN
3,Afghanistan,AFG,1953,28.9304,1240.0,NaN,7764549.0,NaN,NaN
4,Afghanistan,AFG,1954,29.2258,1245.0,NaN,7864289.0,NaN,NaN


In [66]:
df2.rename(columns={'Entity': 'Country'}, inplace=True)
df2.rename(columns={'Period life expectancy at birth - Sex: all - Age: 0': 'Life expectancy'}, inplace=True)
years_to_keep = [2018]
df2 = df2[df2['Year'].isin(years_to_keep)]
regions_to_keep = ['Low-income countries','High-income countries']
df2 = df2[df2['Country'].isin(regions_to_keep)]
print(df2['Country'].unique())
print(df2['Year'].unique())

['High-income countries' 'Low-income countries']
[2018]


In [67]:
# Replace 'Country' with the actual column name in your DataFrame
custom_order = ['Low-income countries', 'High-income countries']

# Convert 'Country' column to categorical with custom order
df2['Country'] = pd.Categorical(df2['Country'], categories=custom_order, ordered=True)

# Sort DataFrame based on the categorical 'Country' column
df2_sorted = df2.sort_values(by='Country')

# Convert the column to string type
df2_sorted['Year'] = df2_sorted['Year'].astype(str)
# Now, df_sorted contains the DataFrame sorted according to the custom order

In [68]:

fig_bar_lifeexp = px.bar(df2_sorted, x="Life expectancy", y="Country", title="Life expectancy in Low-income vs. High-income countries", orientation='h')

# Update traces to hide legend
fig_bar_lifeexp.update_traces(showlegend=False)

fig_bar_lifeexp.update_layout(
    margin=dict(b=0, r=3),
                              xaxis_title_font=dict(size=18),
                             yaxis_title_font=dict(size=18),
                             xaxis=dict(tickfont=dict(size=14)),
                             yaxis=dict(tickfont=dict(size=14)),
    title_font=dict(size=20),
height = 500
)
fig_bar_lifeexp.update_xaxes(title_text="")
# Update label font size on bars
fig_bar_lifeexp.update_traces(textfont_size=24)  # Adjust size as needed
fig_bar_lifeexp.show()

In [69]:
df2_sorted

,Country,Code,Year,Life expectancy,GDP per capita,417485-annotations,Population (historical estimates),Continent,"Mortality rate, under-5 (per 1,000 live births)"
31608,Low-income countries,NaN,2018,62.9042,NaN,NaN,6.434208e+08,NaN,NaN
23163,High-income countries,NaN,2018,80.9854,NaN,NaN,1.209045e+09,NaN,NaN


In [70]:
fig_percent2 = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = 63,
    number ={"suffix":" Years"},
    mode = "gauge+delta+number",
    delta={'reference': 81, 'valueformat': '.0f', 'increasing.color': 'red'},
    title = {'text': "Life expectancy of low-income countries",
         'font': {'size': 25}},
    gauge = {'axis': {'range': [0, 90]},
                 'steps' : [
                 {'range': [45, 67.5], 'color': "lightgray"},
                 {'range': [67.5, 90], 'color': "gray"}],
              'threshold' : {'line': {'color': "lime", 'width': 5}, 'thickness': 0.75, 'value': 81},
             
             'bar':{'color':'red', 'thickness': 0.5}}))
fig_percent2.update_layout(
                            margin=dict(t=0, b=0),
height = 380)


fig_percent2.show()

In [71]:
fig_percent3 = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = 81,
    number ={"suffix":" Years"},
    mode = "gauge+number",
    title = {'text': "Life expectancy of high-income countries",
         'font': {'size': 25}},
    gauge = {'axis': {'range': [0, 90]},
                 'steps' : [
                 {'range': [45, 67.5], 'color': "lightgray"},
                 {'range': [67.5, 90], 'color': "gray"}],
              'threshold' : {'line': {'color': "lime", 'width': 5}, 'thickness': 0.75, 'value': 81},
             
             'bar':{'color':'lime', 'thickness': 0.5}}))
fig_percent3.update_layout(
                            margin=dict(t=0, b=0),
height=380)


fig_percent3.show()

## Figure 4 - Pictoral chart to show difference in child morality


## Implementing child mortality statistic

In [72]:
df3 = merged_df.copy()

In [73]:
df3.dropna(subset=['Mortality rate, under-5 (per 1,000 live births)'])

,Entity,Code,Year,Period life expectancy at birth - Sex: all - Age: 0,GDP per capita,417485-annotations,Population (historical estimates),Continent,"Mortality rate, under-5 (per 1,000 live births)"
10,Afghanistan,AFG,1960,32.5346,1326.0000,NaN,8622473.0,NaN,357.3
11,Afghanistan,AFG,1961,33.0681,1309.0000,NaN,8790140.0,NaN,351.7
12,Afghanistan,AFG,1962,33.5471,1302.0000,NaN,8969055.0,NaN,345.8
13,Afghanistan,AFG,1963,34.0162,1298.0000,NaN,9157463.0,NaN,340.2
14,Afghanistan,AFG,1964,34.4942,1291.0000,NaN,9355510.0,NaN,334.8
...,...,...,...,...,...,...,...,...,...
63177,Zimbabwe,ZWE,2017,60.7095,1582.3662,NaN,14751101.0,NaN,56.2
63178,Zimbabwe,ZWE,2018,61.4141,1611.4052,NaN,15052191.0,NaN,53.7
63179,Zimbabwe,ZWE,2019,61.2925,NaN,NaN,15354606.0,NaN,52.7
63180,Zimbabwe,ZWE,2020,61.1242,NaN,NaN,15669663.0,NaN,51.8


In [74]:
# Subset the DataFrame to include only rows where the year is 2018
df3 = df3[df3['Year'] == 2018]

In [75]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 278 entries, 68 to 63178
Data columns (total 9 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   Entity                                               278 non-null    object 
 1   Code                                                 240 non-null    object 
 2   Year                                                 278 non-null    int64  
 3   Period life expectancy at birth - Sex: all - Age: 0  257 non-null    float64
 4   GDP per capita                                       178 non-null    float64
 5   417485-annotations                                   1 non-null      object 
 6   Population (historical estimates)                    254 non-null    float64
 7   Continent                                            0 non-null      object 
 8   Mortality rate, under-5 (per 1,000 live births)      173 non-null   

In [76]:
df3_poor = df3[df3['GDP per capita'] <= 1000]
df3_poor.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 8400 to 39722
Data columns (total 9 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   Entity                                               5 non-null      object 
 1   Code                                                 5 non-null      object 
 2   Year                                                 5 non-null      int64  
 3   Period life expectancy at birth - Sex: all - Age: 0  5 non-null      float64
 4   GDP per capita                                       5 non-null      float64
 5   417485-annotations                                   0 non-null      object 
 6   Population (historical estimates)                    5 non-null      float64
 7   Continent                                            0 non-null      object 
 8   Mortality rate, under-5 (per 1,000 live births)      4 non-null     

In [77]:
df3_rich = df3[df3['GDP per capita'] >= 60000]
df3_rich.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7 entries, 25499 to 58832
Data columns (total 9 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   Entity                                               7 non-null      object 
 1   Code                                                 7 non-null      object 
 2   Year                                                 7 non-null      int64  
 3   Period life expectancy at birth - Sex: all - Age: 0  7 non-null      float64
 4   GDP per capita                                       7 non-null      float64
 5   417485-annotations                                   0 non-null      object 
 6   Population (historical estimates)                    7 non-null      float64
 7   Continent                                            0 non-null      object 
 8   Mortality rate, under-5 (per 1,000 live births)      7 non-null    

In [78]:
df3_poor.head()

,Entity,Code,Year,Period life expectancy at birth - Sex: all - Age: 0,GDP per capita,417485-annotations,Population (historical estimates),Continent,"Mortality rate, under-5 (per 1,000 live births)"
8400,Burundi,BDI,2018,61.6884,651.3589,NaN,11493476.0,NaN,59.1
9747,Central African Republic,CAF,2018,54.3693,623.4889,NaN,5094795.0,NaN,108.8
13394,Democratic Republic of Congo,COD,2018,59.9417,859.3817,NaN,87087352.0,NaN,NaN
30572,Liberia,LBR,2018,60.8532,818.4211,NaN,4889396.0,NaN,82.6
39722,Niger,NER,2018,62.4544,964.6601,NaN,22577060.0,NaN,117.7


In [79]:
##average mortality for poor
average_mortality_rate_poor = df3_poor['Mortality rate, under-5 (per 1,000 live births)'].mean()
print("Average mortality rate for poor countries =", average_mortality_rate_poor)
#average mortality for rich
average_mortality_rate_rich = df3_rich['Mortality rate, under-5 (per 1,000 live births)'].mean()
print("Average mortality rate for rich countries =", average_mortality_rate_rich)

Average mortality rate for poor countries = 92.05
Average mortality rate for rich countries = 4.957142857142857


In [80]:
##how many times more
timesmore= average_mortality_rate_poor / average_mortality_rate_rich
print("Poor children below 5 are", timesmore, "times more likely to die than those from rich countries")

Poor children below 5 are 18.569164265129682 times more likely to die than those from rich countries


In [81]:
#Using Pillow to read the the image
pil_img = Image.open("childmortimage2.png")

In [82]:
### visualise using an image
image_path = "childmortimage2.png"
card_child_mort = dbc.Card(
    [
        dbc.CardHeader("Children are much more likely to die in low-income countries than high-income countries.",
                       style={"background-color": "Black", "color": "White", "fontSize":"30px",'border-bottom':'groove'}),
        dbc.CardBody(
            [
                html.Div(id='childmortstat', className='card-value text-center'),
                dbc.CardImg(src=pil_img, top=True)
            ]
        )
    ], style={'border':'groove'}, outline=True
)

# Figure 5 - Different resulting life expectancy increase from same GDP increase - bar chart

In [83]:

# Define the coefficients and constant term
coefficient = 12.6247
constant = 21.8567


# Define the GDP per capita for which you want to predict life expectancy
gdp_per_capita_low = 2000  # Example GDP per capita value

# Calculate the predicted life expectancy
log_gdp_per_capita = math.log10(gdp_per_capita_low)
predicted_life_expectancy_low = coefficient * log_gdp_per_capita + constant

print("Predicted Life for 1000GDP Expectancy:", predicted_life_expectancy_low)

# Define the GDP per capita for which you want to predict life expectancy
gdp_per_capita_lowmedium = 25000  # Example GDP per capita value

# Calculate the predicted life expectancy
log_gdp_per_capita = math.log10(gdp_per_capita_lowmedium)
predicted_life_expectancy_lowmedium = coefficient * log_gdp_per_capita + constant

print("Predicted Life for 10000GDP Expectancy:", predicted_life_expectancy_lowmedium)

realincrease_lowtomediumlow = predicted_life_expectancy_lowmedium - predicted_life_expectancy_low
###calculate percentage increase
lowtomedium_percentageinc = ((predicted_life_expectancy_lowmedium - predicted_life_expectancy_low) / predicted_life_expectancy_low)*100
##print results
print("Real increase of Life expectancy from increasing gdp from",gdp_per_capita_low,"to", gdp_per_capita_lowmedium,":", realincrease_lowtomediumlow)
print("Percentage increase of Life expectancy from increasing gdp from",gdp_per_capita_low,"to", gdp_per_capita_lowmedium,":", lowtomedium_percentageinc)


Predicted Life for 1000GDP Expectancy: 63.53121338625907
Predicted Life for 10000GDP Expectancy: 77.37937322748188
Real increase of Life expectancy from increasing gdp from 2000 to 25000 : 13.84815984122281
Percentage increase of Life expectancy from increasing gdp from 2000 to 25000 : 21.79741123001749


In [84]:
# Define the GDP per capita for which you want to predict life expectancy
gdp_per_capita_medium = 48000  # Example GDP per capita value

# Calculate the predicted life expectancy
log_gdp_per_capita = math.log10(gdp_per_capita_medium)
predicted_life_expectancy_medium = coefficient * log_gdp_per_capita + constant

print("Predicted Life for 10000GDP Expectancy:", predicted_life_expectancy_medium)
#calculate real increase
realincrease_mediumtomediumlow = predicted_life_expectancy_medium - predicted_life_expectancy_lowmedium
###calculate percentage increase
lowmediumtomedium_percentageinc = ((predicted_life_expectancy_medium - predicted_life_expectancy_lowmedium) / predicted_life_expectancy_lowmedium)*100
##print results
print("Percentage increase of Life expectancy from increasing gdp from",gdp_per_capita_lowmedium,"to", gdp_per_capita_medium,":", realincrease_mediumtomediumlow)
print("Real increase of Life expectancy from increasing gdp from",gdp_per_capita_lowmedium,"to", gdp_per_capita_medium,":", lowmediumtomedium_percentageinc)


Predicted Life for 10000GDP Expectancy: 80.95596624949559
Percentage increase of Life expectancy from increasing gdp from 25000 to 48000 : 3.5765930220137108
Real increase of Life expectancy from increasing gdp from 25000 to 48000 : 4.622153001290345


In [85]:
import pandas as pd

# Create a list of lists containing the data
data = [
    ['Income Category', 'Life expectancy increase'],  # Add a comma here
    ['Low-income countries', 13.85],
    ['Middle-income countries', 3.58]
]

# Extract column names from the first row
columns = data[0]

# Create a DataFrame without the header
df5 = pd.DataFrame(data[1:], columns=columns)
print(df5)


           Income Category  Life expectancy increase
0     Low-income countries                     13.85
1  Middle-income countries                      3.58


In [86]:
import plotly.express as px

# Create the bar plot
fig_lifeexpincrease_bar = px.bar(df5, x="Income Category", y="Life expectancy increase", orientation='v',
                          title="Increase in life-expectancy when GDP per<br>capita is increased by $13000",
                          labels={"Life expectancy increase": "Estimated life expectancy increase (Years)", "Income Category": ""})

# Update bar colors
#fig_lifeexpincrease_bar.update_traces(marker=dict(color=['green', 'red']))

# Update title size
fig_lifeexpincrease_bar.update_layout(title_font_size=22, height=550)

# Update axis titles size and orientation
fig_lifeexpincrease_bar.update_layout(xaxis=dict(title_font_size=20, tickfont_size=20), 
                              yaxis=dict(title_font_size=20, title_standoff=20, tickangle=0, tickfont_size=20))

# Hide legend
fig_lifeexpincrease_bar.update_layout(showlegend=False)

# Update label font size on bars
fig_lifeexpincrease_bar.update_traces(textfont_size=24)  # Adjust size as needed

# Show the plot
fig_lifeexpincrease_bar.show()


## Integrating into dashboard

In [87]:
import dash_bootstrap_components as dbc
# Define the dropdown options
dropdown_options = [
    {'label': 'Selected Value: Life Expectancy Overtime', 'value': 'Life expectancy'},
    {'label': 'Selected Value: Mortality Rate Under-5 Overtime', 'value': 'Mortality rate, under-5 (per 1,000 live births)'},
    {'label': 'Selected Value: GDP per Capita Overtime', 'value': 'GDP per capita'}
]


In [88]:
##First card
card_scatter_lifegdp = dbc.Card(
    [
        dbc.CardHeader("Life expectancy increases as GDP increases at a diminishing rate",
                       style={"background-color": "Black", "color": "White", "fontSize":"30px",'border-bottom':'groove'}),
        dbc.CardBody(
            [
                html.Div(id='global_lifeexp', className='card-value text-center'),
                dcc.Graph(figure=fig_scatter)
            ]
        )
    ], style={'border':'groove'}
)


###gauge chart cards
card_gauge_1 = dbc.Card(
    [
        #dbc.CardHeader('Citizens in Low-Income Countries Die 18 Years Earlier Than in High-Income Countries.',style={"background-color": "White", "color": "Black", "fontSize":"20px"}),
        dbc.CardBody(
            [
                html.Div(id='fig_gauge_1', className='card-value text-center'),
                dcc.Graph(figure=fig_percent3)
            ]
        )
    ], style={'border':'groove'}, outline=True
)

card_gauge_2 = dbc.Card(
    [
        #dbc.CardHeader('Citizens in Low-Income Countries Die 18 Years Earlier Than in High-Income Countries.',style={"background-color": "White", "color": "Black", "fontSize":"20px"}),
        dbc.CardBody(
            [
                html.Div(id='fig_gauge_2', className='card-value text-center'),
                dcc.Graph(figure=fig_percent2)
            ]
        )
    ], style={'border':'groove'}, outline=True
)
card_gauge_3 = dbc.Card(
    [
        dbc.CardHeader('People in Low-Income Countries Die 18 Years Earlier Than those in High-Income Countries.',
                       style={"background-color": "Black", "color": "White", "fontSize": "30px",'border-bottom':'groove'}),
        dbc.CardBody(
            [
                html.Div(id='fig_gauge_3', className='card-value text-center'),
                dbc.Row([
                    dbc.Col(dcc.Graph(figure=fig_percent3), width=6),
                    dbc.Col(dcc.Graph(figure=fig_percent2), width=6),
                ])
            ]
        )
    ], style={'border': 'groove'}, outline=True
)

##"Third" card
card_global_map = dbc.Card(
    [
        dbc.CardHeader('Socioeconomic values have increased overtime, but at an inequal rate',
                       style={"background-color": "Black", "color": "White", "fontSize":"30px",'border-bottom':'groove'}),
        dbc.CardBody(
            [
                html.Div(id='global_unemp', className='card-value text-center'),
                dbc.Row([
                    dbc.Col(html.Div("Global Socioeconomic Values Overtime", style={'font-size': '30px', 'font-weight':'bold'}), width=6),
                    dbc.Col(dcc.Dropdown(
                        id='color-dropdown',
                        options=dropdown_options,
                        clearable=False,
                        value='Life expectancy',  # Default value for the dropdown
                        style={'width': '85%', 'z-index':'1000', 'font-weight':'bold'}), width=6),
                ]),
                dcc.Graph(id='world-map')  # Use id='world-map' to reference in the callback
            ]
        )
    ], style={'border':'groove'}, outline=True
)



#"Second" card
card_figincreasebar = dbc.Card(
    [
        dbc.CardHeader("Poor countries benefit significantly more from an increase in GDP than richer countries",
                       style={"background-color": "Black", "color": "White", "fontSize":"30px",'border-bottom':'groove'}),
        dbc.CardBody(
            [
                html.Div(id='fig_lifeexpincrease_bar', className='card-value text-center'),
                dcc.Graph(figure=fig_lifeexpincrease_bar),
            ]
        )
    ], style={'border':'groove'}, outline=True
)


## Initalize Dash

In [89]:
### exeter logo
exe_logo = Image.open("exeterbuswhite.png")

In [90]:
# Initialize the app with Bootstrap theme
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Your DataFrame 'df' should be defined here
df = merged_df.copy()
df.rename(columns={'Entity': 'Country'}, inplace=True)
df.rename(columns={'Period life expectancy at birth - Sex: all - Age: 0': 'Life expectancy'}, inplace=True)
# Remove rows with empty values in column 'Code'
df = df.dropna(subset=['Code'])
# Filter out rows where the year is below 1950
df = df[df['Year'] >= 1950]
# Drop rows where the country is 'Qatar' - qatar is an outlier and reduces visibility on graph.
df = df[df['Country'] != 'Qatar']
# Reset index after filtering
df.reset_index(drop=True, inplace=True)

# Define the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define the layout of the app
app.layout = html.Div([
    html.Div([
        #html.Div(html.Img(src=exe_logo, height="100px"), style={'width': '250px','background-color':'black'}),  # Insert the logo here and set a fixed width
        html.H1("Global Health Inequality Analytical Dashboard", style={'flex': '1',
                                                                               'padding': '10px 0 10px 10px',
                                                                               'text-align': 'left',
                                                                               'font-size':'64px',
                                                                               'color':'white'}),  # Set flex: 1 to the header text to push it to the center
        html.Div(html.Img(src=exe_logo, height="100px"), style={'width': '250px','background-color':'black'}),
    ], style={'display': 'flex', 'align-items': 'left', 'background-color': 'black', 'width': '100%','border-bottom':'10px solid White'}),  # Removed justify-content: center
    dbc.Row([
        dbc.Col(card_child_mort, width=6),
        #dbc.Col(card_gauge_1, width=3),
        dbc.Col(card_gauge_3, width=6)
    ]),
    dbc.Row([
        dbc.Col(card_global_map, width=5),
        dbc.Col(card_scatter_lifegdp, width=4),
        dbc.Col(card_figincreasebar, width=3)
    ])
])
df_lifeexp = df.copy()
df_gdp = df[df['Year'] <= 2018]
df_childmort = df[(df['Year'] >= 1960) & (df['Year'] <= 2018)]
# Define callback to update the map based on the dropdown selection
@app.callback(
    Output('world-map', 'figure'),
    [Input('color-dropdown', 'value')]
)

def update_map(selected_column):
    if selected_column == 'Life expectancy':
        colour='Hot'
        minimum = 40
        maximum = 90
        selected = df_lifeexp
    elif selected_column =='GDP per capita':
        colour='Blackbody'
        minimum = None
        middle = None
        maximum = None
        selected = df_gdp
    else:
        colour='Hot_r'
        minimum = 0
        middle = 50
        maximum = 100
        selected = df_childmort
    fig_map = px.choropleth(selected,
                            locations="Country",
                            locationmode='country names',
                            color=selected_column,
                            range_color=(minimum, maximum),
                            hover_name="Country",
                            hover_data={"Life expectancy": True, "GDP per capita": True,
                                        "Mortality rate, under-5 (per 1,000 live births)": True},
                            color_continuous_scale=colour,
                            labels={'number': selected_column},
                            animation_frame='Year')

    # Update layout of the map
    fig_map.update_layout(
        geo=dict(showframe=True, showcoastlines=True),
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        margin=dict(l=2, r=10, t=0, b=2),  # Reduce margins
        coloraxis_colorbar=dict(len=0.75, x=0.5, y=-0.15,
                                orientation='h',outlinecolor='Black',outlinewidth=1),
        coloraxis_colorbar_ticks="outside",  # Place colorbar ticks outside the colorbar
        coloraxis_colorbar_tickfont=dict(size=12),  # Adjust colorbar tick font size
        legend=dict(
            title=dict(
                font=dict(size=22)),
        font=dict(size=22)),
    height=600, width=1000,  # Adjust plot height
        sliders=dict(
            x=0.1,  # Adjust x position of the slider (0 is left, 1 is right)
            y=0.5,  # Adjust y position of the slider (0 is bottom, 1 is top)
        )
    )
    if selected_column == 'Mortality rate, under-5 (per 1,000 live births)':
        fig_map.update_layout(
        coloraxis_colorbar=dict(
            tickmode='array',
            tickvals=[0,50,100],
            ticktext=['0','50','>100']))
    fig_map.update_layout(
    coloraxis_colorbar_tickfont=dict(size=15))

    return fig_map

# Run the app
if __name__ == '__main__':
    app.run_server(jupyter_mode="external", debug=True, port=8054)

Dash app running on http://127.0.0.1:8054/
